In [1]:

import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Conv2D
from keras.layers import Conv1D
from keras.layers import Masking
from sklearn.preprocessing import normalize
from keras.models import load_model
from keras.utils import multi_gpu_model

import keras
import pickle
import numpy as np
from tensorflow.python.client import device_lib
from keras import backend as K
from keras.utils import to_categorical

print(device_lib.list_local_devices()) # list of DeviceAttributes

# %gui qt
import numpy as np
import mne
import pickle
import sys
import os
import matplotlib

import matplotlib.pyplot as plt

# import vispy
# print(vispy.sys_info())
# BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
# sys.path.append(BASE_DIR)
%matplotlib inline
mne.utils.set_config('MNE_USE_CUDA', 'true')  
mne.cuda.init_cuda(verbose=True)

Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10061954939665648104
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 11621314086826064725
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 621215744
locality {
  bus_id: 1
  links {
  }
}
incarnation: 735452637056015375
physical_device_desc: "device: 0, name: TITAN RTX, pci bus id: 0000:65:00.0, compute capability: 7.5"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 7724187648
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5978883732378244276
physical_device_desc: "device: 1, name: GeForce RTX 2070 SUPER, pci bus id: 0000:17:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 11217499458353241581
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:1"
device_type: "XLA_GPU"

In [2]:
baseFolder='./pickled-avg'
files=[f for f in os.listdir(baseFolder) if not f.startswith('.')]

data=pickle.load(open('pickled-avg/OpenBCISession_2020-03-06_10-28-17-LUKE', 'rb'))

In [3]:
data[1]

[9,
 26244.93,
 2880.89,
 1376.11,
 14922.76,
 7416.26,
 18995.14,
 9926.9,
 12764.05,
 25360.13,
 -12.83,
 -9006.39,
 9666.1,
 24227.99,
 18975.42,
 21420.93,
 15267.38,
 504,
 datetime.datetime(1900, 1, 1, 10, 29, 37, 406000),
 {'bpm': -1,
  'ibi': -1,
  'sdnn': -1,
  'sdsd': -1,
  'rmssd': -1,
  'pnn20': -1,
  'pnn50': -1,
  'hr_mad': -1,
  'sd1': -1,
  'sd2': -1,
  's': -1,
  'sd1/sd2': -1,
  'breathingrate': -1,
  'lf': -1,
  'hf': -1,
  'lf/hf': -1,
  'segment_indices': (-1, -1)},
 'sync']

In [4]:

hot= {
    'paced':0,
    'slowBreath':1,
    'rest':2,
    'baseline':3,
    'sync':4,
    'survey':5,
    'stressor':6
}

In [5]:
good={
    'paced':True,
    'slowBreath':True,
    'rest':False,
    'baseline':True,
    'sync':False,
    'survey':False,
    'stressor':True
}

In [6]:
features=[]
labels=[]
add=[]
prev='sync'
for i in range(len(data)):
    if good[data[i][20]]:
#         print(data[i][20])
        if prev!=data[i][20] :
            print(prev, data[i][20], len(add))
            features.append(add)
            labels.append(hot[data[i-1][20]])
            add = []


        add.append(data[i][1:17])
    prev=data[i][20]
features=features[1:]
labels=labels[1:]
#     try:
#         features.append(data[i][19]['bpm'])
    
#     except Exception as e: 
#         print(e, i)
#         print(data[i])

#     for k in range(1, 17):
#         add.append([data[i][k]])
#     add.append([data[0][19]['bpm']])
#     labels.append([float(hot[data[i][20]])])

sync baseline 0
baseline stressor 15030
rest slowBreath 11302
rest stressor 25070
rest paced 11241
rest stressor 25015
rest slowBreath 11242
rest stressor 25066
rest paced 11302


In [7]:
features[0][0]

[26095.91,
 2745.84,
 1289.29,
 14885.05,
 7387.41,
 18965.54,
 9847.4,
 12733.61,
 25351.04,
 11.53,
 -9099.06,
 9614.27,
 24143.64,
 18917.09,
 21363.39,
 15275.2]

In [8]:
labels

[3, 2, 2, 2, 2, 2, 2, 2]

In [9]:
# train_X = np.array(features[0:int(7*len(features)/10)])
# train_y = np.array(labels[0:int(7*len(labels)/10)])

# test_X = np.array(features[int(7*len(features)/10):len(features)])
# test_y = np.array(labels[int(7*len(labels)/10):len(labels)])

train_X = np.array(features)
train_y = np.array(labels)

test_X = np.array(features)
test_y = np.array(labels)

In [10]:
train_y = to_categorical(train_y)
test_y = to_categorical(test_y)

In [11]:
# scale=train_y[0]/(normalize(train_y.reshape(-1, 1), axis=0)[0])
# scale=scale[0]
# print(train_y[0], (normalize(train_y.reshape(-1, 1), axis=0)[0][0]))
# print("Scale factor is", scale)

In [12]:
print("train_X shape is", train_X.shape)
print("train_y shape is", train_y.shape)
print("test_X shape is", test_X.shape)
print("test_y shape is", test_y.shape)


train_X shape is (8,)
train_y shape is (8, 4)
test_X shape is (8,)
test_y shape is (8, 4)


In [16]:
train_X=keras.preprocessing.sequence.pad_sequences(train_X, maxlen=None, dtype='float32', padding='pre', truncating='pre', value=-9999)
test_X=keras.preprocessing.sequence.pad_sequences(test_X, maxlen=None, dtype='float32', padding='pre', truncating='pre', value=-9999)
# train_y=normalize(train_y.reshape(-1, 1), axis=0)
# test_y=normalize(test_y.reshape(-1, 1), axis=0)

# design network

In [ ]:
model = Sequential()
model.add(Masking(mask_value=-9999, input_shape=(None, test_X.shape[-1])))
model.add(LSTM(220))
model.add(Dense(50, activation='sigmoid'))
model.add(Dense(20, activation='relu'))
model.add(Dense(train_y.shape[1], activation='softmax'))
# model = multi_gpu_model(model)
model.compile(loss='mae', optimizer='adam')
# fit network

history = model.fit(train_X, train_y, epochs=100, batch_size=125, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 8 samples, validate on 8 samples
Epoch 1/100
 - 16s - loss: 0.3228 - val_loss: 0.2966
Epoch 2/100
 - 16s - loss: 0.2966 - val_loss: 0.2734
Epoch 3/100
 - 16s - loss: 0.2734 - val_loss: 0.2506
Epoch 4/100
 - 16s - loss: 0.2506 - val_loss: 0.2277
Epoch 5/100
 - 16s - loss: 0.2277 - val_loss: 0.2072
Epoch 6/100
 - 16s - loss: 0.2072 - val_loss: 0.1883
Epoch 7/100
 - 16s - loss: 0.1883 - val_loss: 0.1720
Epoch 8/100
 - 16s - loss: 0.1720 - val_loss: 0.1574
Epoch 9/100
 - 16s - loss: 0.1574 - val_loss: 0.1444
Epoch 10/100
 - 16s - loss: 0.1444 - val_loss: 0.1330
Epoch 11/100
 - 16s - loss: 0.1330 - val_loss: 0.1231
Epoch 12/100
 - 16s - loss: 0.1231 - val_loss: 0.1146
Epoch 13/100
 - 16s - loss: 0.1146 - val_loss: 0.1074
Epoch 14/100
 - 16s - loss: 0.1074 - val_loss: 0.1014
Epoch 15/100
 - 16s - loss: 0.1014 - val_loss: 0.0962
Epoch 16/100
 - 16s - loss: 0.0962 - val_loss: 0.0919
Epoch 17

In [ ]:
model.save('./models/eeg-block-2.h5')

In [15]:
test_X.shape

(8,)

In [17]:
# make a prediction
model = load_model('./models/eeg-block.h5')
yhat = model.predict(test_X)
print(yhat)


[[0.02238274 0.00486007 0.91967905 0.05307809]
 [0.02244086 0.00487879 0.9196637  0.05301662]
 [0.02295491 0.00511418 0.9192292  0.05270174]
 [0.02235947 0.00494177 0.9228301  0.04986858]
 [0.01851877 0.00375323 0.9356798  0.04204822]
 [0.01851877 0.00375323 0.9356798  0.04204822]
 [0.01851877 0.00375323 0.9356798  0.04204822]
 [0.01772436 0.00365375 0.9352487  0.04337322]]


In [ ]:
# test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)